# yield from で複数のジェネレータを作る

画面表示プログラムでジェネレータを使ってイメージの動きをアニメーションで示すことを考える

In [25]:
# スクリーン上での動きを示す動き幅は yield する2つのジェネレータ

def move(period, speed):
  for _ in range(period):
    yield speed

def pause(delay):
  for _ in range(delay):
    yield 0

In [26]:
# 最終的なアニメーションのために、
# move と pause を組み合わせて、スクリーン上の移動幅のシーケンスを作る

def animate():
  for delta in move(4, 5.0): # move から値を一個ずつ取り出して
    yield delta              # animate がひとつずつ返す
  for delta in pause(3):
    yield delta
  for delta in move(2, 3.0):
    yield delta

In [27]:
def render(delta):
  print(f'Delta: {delta:.1f}')
  # イメージをスクリーン上で動かす

def run(func):
  for delta in func():
    render(delta)

run(animate)

Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 0.0
Delta: 0.0
Delta: 0.0
Delta: 3.0
Delta: 3.0


このコードの問題は繰り返しにある。とても読みずらい

In [28]:
# 解決策
# yield from 式を使う
# 全部終わるまで yield from で指定したジェネレータから yeild される値を流し続ける
# ---親は yield された値を 1個ずつ見張らない。子ジェネレータに全部丸投げ。---

def animate_composed():
  # 入れ子になったジェネレータ(move / pause)が親のジェネレータ(animate_composed)に制御を戻す前に、
  # 入れ子になった子ジェネレータ(move / pause)からすべての値を生成できる
  yield from move(4, 5.0) # move が返すものをそのまま渡す
  yield from pause(3)
  yield from move(2, 3.0)

run(animate_composed)

Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 0.0
Delta: 0.0
Delta: 0.0
Delta: 3.0
Delta: 3.0


run() 

↓

animate_composed()

├─ yield from move()    (moveが全部出し終わるまで待つ)

├─ yield from pause()   (pauseが全部出し終わるまで待つ)

└─ yield from move()    (moveが全部出し終わるまで待つ)

1. run は animate_composed を呼び出す

1. animate_composed は move を呼び出して、その出力を yield from で全部流す

1. そのあと pause の出力を全部流す

1. そのあとまた move の出力を全部流す

親子関係で言えば、

1. animate_composed（親）が

1. move と pause（子）に仕事を渡して、

1. 子が終わるまで待つ、という感じ。

In [30]:
# yield from を使うと性能も上がる

import timeit

def child():
  for i in range(1_000_000):
    yield i

def slow():
  for i in child():
    yield i

def fast():
  yield from child()

baseline = timeit.timeit(
  stmt = 'for _ in slow(): pass',
  globals = globals(),
  number=50
)

print(f'Manual nesting {baseline:.2f}s')

comparison = timeit.timeit(
  stmt = 'for _ in fast(): pass',
  globals = globals(),
  number=50
)

print(f'Composed nesting {comparison:.2f}s')

reduction = -(comparison - baseline) / baseline
print(f'{reduction:.1%} less time')

Manual nesting 2.26s
Composed nesting 2.33s
-3.2% less time


できる限り yield from を使うことがオススメ